# Lab 04 - ChatGPT - Managing conversation history with CHatGPT
This sample notebook demonstrates a couple of simple patterns you can use for managing the prompts and conversation history with the ChatGPT model.

In [ ]:
pip install tiktoken

In [ ]:
import os
import openai
import json

In [ ]:
# Setting up the deployment name
deployment_name = "chatgpt-turbo"

# This is set to `azure`
openai.api_type = "azure"
openai.api_version = "2022-12-01"

# The API key for your Azure OpenAI resource.
API_KEY = "<USE_YOUR_KEY>"
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = "https://openai-bootcamp-mma.openai.azure.com/"
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

## 1.0 Create the system message for ChatGPT

In [ ]:
base_system_message = """
You are a marketing writing assistant. You help come up with creative content ideas and content like marketing emails, blog posts, tweets, ad copy, listicles, product FAQs, and product descriptions. 
You write in a friendly yet professional tone and you can tailor your writing style that best works for a user-specified audience. 

Additional instructions:
- Make sure you understand your user's audience so you can best write the content.
- Ask clarifying questions when you need additional information. Examples include asking about the audience or medium for the content.
- Don't write any content that could be harmful.
- Don't write any content that could be offensive or inappropriate.
- Don't write any content that speaks poorly of any product or company.
"""

system_message = f"<|im_start|>system\n{base_system_message.strip()}\n<|im_end|>"
print(system_message)

## 2.0 Define helper functions



In [ ]:
# Defining a function to create the prompt from the system message and the messages
# The function assumes `messages` is a list of dictionaries with `sender` and `text` keys
# Example: messages = [{"sender": "user", "text": "I want to write a blog post about my company."}]
def create_prompt(system_message, messages):
    prompt = system_message
    for message in messages:
        prompt += f"\n<|im_start|>{message['sender']}\n{ message['text']}\n<|im_end|>"
    prompt += "\n<|im_start|>assistant\n"
    return prompt

In [ ]:
import tiktoken 

# Defining a function to estimate the number of tokens in a prompt
def estimate_tokens(prompt):
    cl100k_base = tiktoken.get_encoding("cl100k_base") 

    enc = tiktoken.Encoding( 
        name="chatgpt",  
        pat_str=cl100k_base._pat_str, 
        mergeable_ranks=cl100k_base._mergeable_ranks, 
        special_tokens={ 
            **cl100k_base._special_tokens, 
            "<|im_start|>": 100264, 
            "<|im_end|>": 100265
        } 
    ) 

    tokens = enc.encode(prompt,  allowed_special={"<|im_start|>", "<|im_end|>"})
    return len(tokens)

# Estimate the number of tokens in the system message. Tokens in the system message will be sent in every request.
token_count = estimate_tokens(system_message)
print("Token count: {}".format(token_count))

In [ ]:
# Defining a function to send the prompt to the ChatGPT model
def send_message(prompt, model_name, max_response_tokens=500):
    response = openai.Completion.create(
        engine=model_name,
        prompt=prompt,
        temperature=0.5,
        max_tokens=max_response_tokens,
        top_p=0.9,
        frequency_penalty=0,
        presence_penalty=0,
        stop=['<|im_end|>']
    )
    return response['choices'][0]['text'].strip()

# Defining a function to print out the conversation in a readable format
def print_conversation(messages):
    for message in messages:
        print(f"[{message['sender'].upper()}]")
        print(message['text'])
        print()

## 3.0 Start the conversation

In [ ]:
# This is the first message that will be sent to the model. Feel free to update this.
user_message = "I want to write a blog post about the impact of AI on the future of work."

In [ ]:
# Create the list of messages. Sender can be either "user" or "assistant"
messages = [{"sender": "user", "text": user_message}]

# Create the full prompt
prompt = create_prompt(system_message, messages)

print(prompt)

In [ ]:
token_count = estimate_tokens(prompt)
print(f"Token count: {token_count}")

In [ ]:
max_response_tokens = 500

response = send_message(prompt, deployment_name, max_response_tokens)
messages.append({"sender": "assistant", "text": response})

print_conversation(messages)

## 4.0 Continue the conversation

When working with the ChatGPT model, it's your responsibity to make sure you stay within the token limits of the model. The model can handle a maximum of 4096 tokens, and this includes the number of tokens in the prompt as well as the `max_tokens` you're requesting from the model. If you exceed these limits, the model will return an error.

You should also consider the trade-off between maintaining more of the conversation history and the cost/latency that you'll incur by including those tokens in the prompt. Shorter prompts are cheaper and faster. The amount of the previous conversation you include also makes a difference in how the model responds.

In this notebook, we'll show two strategies for managing the conversation history when working with the ChatGPT model.
- Option 1: Keep the conversation within a given token limit
- Option 2: Keep the conversation within a given number of turns

### Option 1: Keep the conversation within a given token limit

`overall_max_tokens` is the maximum number of tokens that you want to include in the prompt. Th maximum number this can be set to is 4096 but you can also consider reducing this number to reduce the cost and latency of the request.

In [ ]:
overall_max_tokens = 4096
prompt_max_tokens = overall_max_tokens - max_response_tokens

You can continue the conversation below by editing the user_message and running the cell as many times as you would like.

In [ ]:
user_message = "The target audience for the blog post should be business leaders working in the tech industry."
#user_message = "Let's talk about generative AI and keep the tone informational but also friendly."
#user_message = "Show me a few more examples"
messages.append({"sender": "user", "text": user_message})

prompt = create_prompt(system_message, messages)
token_count = estimate_tokens(prompt)
print(f"Token count: {token_count}")

# remove first message while over the token limit
while token_count > prompt_max_tokens:
    messages.pop(0)
    prompt = create_prompt(system_message, messages)
    token_count = estimate_tokens(prompt)

response = send_message(prompt, deployment_name, max_response_tokens)

messages.append({"sender": "assistant", "text": response})
print_conversation(messages)

### Option 2: Keep the conversation within a given number of turns

In [ ]:
max_messages = 10

overall_max_tokens = 4096
prompt_max_tokens = overall_max_tokens - max_response_tokens

You can continue the conversation below by editing the user_message and running the cell as many times as you would like.

In [ ]:
user_message = "Make the post about generative AI aimed at business leaders who have some knowledge of the technology."
messages.append({"sender": "user", "text": user_message})

while len(messages) > max_messages:
    messages.pop(0)

prompt = create_prompt(system_message, messages)
token_count = estimate_tokens(prompt)

while token_count > prompt_max_tokens:
    # remove first message from messages
    messages.pop(0)
    prompt = create_prompt(system_message, messages, max_response_tokens)
    token_count = estimate_tokens(prompt)

response = send_message(prompt, deployment_name)
messages.append({"sender": "assistant", "text": response})
# print_conversation(messages)
